# COSC 4337 - Homework 3: RNNs for Time Series Forecasting (Netflix Stocks) 📈

**Objective:** The goal of this assignment is to build, train, and evaluate Recurrent Neural Networks (RNNs), including GRU and LSTM variants, for predicting Netflix stock prices.

***Note on Frameworks***

This notebook uses `Keras` with a `TensorFlow` backend. If you are more comfortable with `PyTorch`, you are welcome to complete the assignment using it. The core concepts are directly transferable. Feel free to change code as you see fit as long as it follows the tasks.

## Problem Statement

You are tasked with predicting the **closing price** of Netflix (NFLX) stock for the next day based on historical daily data.

**Dataset:**
We will use a dataset containing historical daily stock prices for Netflix (NFLX), including Open, High, Low, Close, Adj Close, and Volume. The specific file used (`NFLX.csv`) covers 2018-2022.

**Dataset Features (Input):**
1.  A sequence of a *look_back* number of past closing prices (or other features in later exercises).

**Dataset Target (Output):**
* The closing price for the *next* trading day.

Link to original dataset source concept: https://www.kaggle.com/datasets/jainilcoder/netflix-stock-price-prediction




## Setup: Import Libraries and Set Seed


In [ ]:
# === Import Libraries ===
import numpy as np
import pandas as pd
from tensorflow import random
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# === Set Seed ===
seed = 4337
np.random.seed(seed)
random.set_seed(seed)

## Data Loading and Initial Exploration


In [ ]:
# === 1. Load Data ===

data_path = 

# YOUR CODE HERE: Load the dataset from the url into a pandas DataFrame named 'df'.
# Make sure to parse the 'Date' column as datetime objects using the 'parse_dates' argument.
df = # ... YOUR CODE ...

# Display the first few rows
print("First 5 rows of the dataset:")
# ... YOUR CODE ...

# Display basic info about the DataFrame
print("\nDataFrame Info:")
# ... YOUR CODE ...


## Select Target Variable and Visualize

We are selecting the 'Close' price as our target variable and plotting it over time.
We need to isolate the variable we want to predict. Plotting helps visualize the trends, seasonality, and potential challenges in the data.

In [ ]:
# === 2a. Select Target Variable ===
# YOUR CODE HERE: Extract the 'Close' column as a numpy array, ensure it's float32, and reshape it to be a 2D array (samples, 1 feature).
# Store this in a variable called 'close_prices'.
close_prices = 

print(f"Shape of close_prices: {close_prices.shape}")

# === 2b. Visualize the Close Price ===
plt.figure(figsize=(14, 7))
# YOUR CODE HERE: Plot the 'Date' column from the DataFrame 'df' on the x-axis
# and the 'close_prices' array on the y-axis.
# Add a title 'Netflix Close Price History', xlabel 'Date', and ylabel 'Close Price USD'.

plt.plot(# ... YOUR CODE ... )
plt.title(# ... YOUR CODE ... )
plt.xlabel(# ... YOUR CODE ... )
plt.ylabel(# ... YOUR CODE ... )
plt.grid(True)
plt.show()

## Scale the Data

Neural networks generally perform better with normalized input data. Scaling prevents features with large values from disproportionately influencing the model's learning process and helps gradients flow more effectively.

In [ ]:
# === 3. Scale the Data ===
# YOUR CODE HERE: Initialize a MinMaxScaler with feature_range=(0, 1).
scaler = # ... YOUR CODE ...

# YOUR CODE HERE: Apply the scaler to 'close_prices' using fit_transform and store the result in 'dataset_scaled'.
dataset_scaled = # ... YOUR CODE ...

print("Shape of scaled dataset:", dataset_scaled.shape)
print("First 5 scaled values:\n", dataset_scaled[:5])

## Define Sequence Creation Function

We are creating a reusable function to transform the time series data into input sequences (X) and target outputs (y).
Define a function `create_sequences` that takes the scaled data and `look_back` period as input. It iterates through the data, creating sequences of length `look_back` and pairing them with the immediately following data point as the target. RNNs require input data structured as sequences. This function automates the process of generating these sequence-target pairs, which is fundamental for training.

In [ ]:
# === 4. Define Sequence Creation Function ===
def create_sequences(data, look_back=1):
    """Converts univariate time series data into sequences for RNNs."""
    X, y = [], []
    for i in range(len(data) - look_back):
        sequence = data[i:(i + look_back), 0]
        target = data[i + look_back, 0]
        X.append(sequence)
        y.append(target)
    return np.array(X), np.array(y)

# Test the function (optional)
test_data = np.arange(10).reshape(-1, 1)
testX, testY = create_sequences(test_data, 3)
print("Test Data:\n", test_data.flatten())
print("Test X (look_back=3):\n", testX)
print("Test Y (look_back=3):\n", testY)

## Create Sequences and Reshape

We are applying the `create_sequences` function and reshaping the resulting input data `X`.
Call the function with the scaled data and the desired `look_back`. Then use `np.reshape` on `X`.
We are doing this to generate the actual training/testing sequences based on our chosen lookback period (e.g., 60 days). The reshaping step is mandatory to get the 3D format `[samples, timesteps, features]` required by Keras RNN layers.

In [ ]:
# === 5a. Create Sequences ===
# We'll use the past 60 days (timesteps) to predict the next day.
look_back = 60

# YOUR CODE HERE: Call the 'create_sequences' function with 'dataset_scaled' and 'look_back'.
# Store the results in variables 'X' and 'y'.
X, y = # ... YOUR CODE ...

print(f"Created {X.shape[0]} sequences.")
print("Shape of X before reshape:", X.shape)
print("Shape of y:", y.shape)

# === 5b. Reshape Input Data for RNN ===
# Reshape X to the required 3D format: [samples, time steps, features]
# YOUR CODE HERE: Reshape 'X' using np.reshape. The new shape should have:
# - X.shape[0] as the number of samples
# - X.shape[1] (which is look_back) as the number of timesteps
# - 1 as the number of features (since we are only using 'Close' price)
X = # ... YOUR CODE ...

print("Shape of X after reshape:", X.shape)

## Split Data into Training and Testing Sets

We are dividing the sequential data into a set for training the model and a separate set for evaluating its performance.
Calculate a split point (e.g., 80% mark) and slicing the `X` and `y` arrays. 
For time series forecasting, the model must learn from the past to predict the future. We train on the earlier portion of the data and test on the later portion to mimic this real-world scenario.

**Crucial to notice that we do not shuffle the data.**

In [ ]:
# === 6. Train/Test Split (Sequential) ===
# Use 80% of the data for training, 20% for testing.
train_size_percentage = 0.80

# YOUR CODE HERE: Calculate the integer index 'train_size' representing the end of the training data.
train_size = # ... YOUR CODE ...

# YOUR CODE HERE: Slice the 'X' and 'y' arrays to create:
# - X_train, y_train (from the beginning up to train_size)
# - X_test, y_test (from train_size to the end)
X_train, X_test = # ... YOUR CODE ...
y_train, y_test = # ... YOUR CODE ...

# --- Verification --- 
print("--- Split Shapes --- ")
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

## Exercise 1: Build and Compare RNN Models

For this task define, compile, and train three different types of recurrent neural networks: SimpleRNN, GRU, and LSTM.

We will compare the performance of these architectures. Stock prices have **long-term dependencies** (e.g., the price 60 days ago can still influence today's trend). 
* **SimpleRNNs** struggle with this due to the **vanishing gradient problem**.
* **LSTMs (Long Short-Term Memory)** use gates (forget, input, output) and a cell state to selectively remember or forget information over long sequences, mitigating the vanishing gradient issue.
* **GRUs (Gated Recurrent Units)** are a simplified version of LSTMs with fewer gates (update, reset), often achieving similar performance with less computation.

### 1a. Define Model Architectures
Specify the layers for each model.
Use `Sequential()` and add the appropriate RNN layer (`SimpleRNN`, `GRU`, `LSTM`) with 32 units, followed by a `Dense` output layer with 1 unit.
This sets up the structure of each network. 32 units is an arbitrary starting point for hidden layer complexity. The `Dense(1)` layer is required for our regression task (predicting one continuous value).

In [ ]:
# === Define Input Shape ===
input_shape = (look_back, 1)

# --- Define Model 1 (SimpleRNN) ---
model_rnn = Sequential(name="SimpleRNN_Model")
# YOUR CODE HERE: Add a SimpleRNN layer with 32 units. Specify the input_shape.
model_rnn.add(# ... YOUR CODE ...
# YOUR CODE HERE: Add a Dense output layer with 1 unit.
model_rnn.add(# ... YOUR CODE ...

# --- Define Model 2 (GRU) ---
model_gru = Sequential(name="GRU_Model")
# YOUR CODE HERE: Add a GRU layer with 32 units. Specify the input_shape.
model_gru.add(# ... YOUR CODE ...
# YOUR CODE HERE: Add a Dense output layer with 1 unit.
model_gru.add(# ... YOUR CODE ...

# --- Define Model 3 (LSTM) ---
model_lstm = Sequential(name="LSTM_Model")
# YOUR CODE HERE: Add an LSTM layer with 32 units. Specify the input_shape.
model_lstm.add(# ... YOUR CODE ...
# YOUR CODE HERE: Add a Dense output layer with 1 unit.
model_lstm.add(# ... YOUR CODE ...

print("Models defined.")

### 1b. Compile Models


In [ ]:
# --- Compile all models ---
print("Compiling models...")
# YOUR CODE HERE: Compile model_rnn with 'adam' optimizer and 'mean_squared_error' loss.
model_rnn.compile(# ... YOUR CODE ...

# YOUR CODE HERE: Compile model_gru similarly.
model_gru.compile(# ... YOUR CODE ...

# YOUR CODE HERE: Compile model_lstm similarly.
model_lstm.compile(# ... YOUR CODE ...

# --- Print Model Summary (Example) ---
print("\n--- Model 3 (LSTM) Summary ---")
model_lstm.summary()

### 1c. Train the Models


 We use `validation_split=0.1` to monitor performance on unseen data during training.

In [ ]:
# Training parameters
epochs = 50 # Number of passes through the entire training dataset.
batch_size = 32 # Number of samples processed before the model's weights are updated.

print(f"Training Model 1 (SimpleRNN) for {epochs} epochs...")
# YOUR CODE HERE: Train 'model_rnn' using .fit().
# Pass X_train, y_train, epochs, batch_size, validation_split=0.1, and verbose=1.
# Store the output history in 'history_rnn'.
history_rnn = # ... YOUR CODE ...


In [ ]:
print(f"\nTraining Model 2 (GRU) for {epochs} epochs...")
# YOUR CODE HERE: Train 'model_gru' similarly to model_rnn.
# Store the output history in 'history_gru'.
history_gru = # ... YOUR CODE ...


In [ ]:
print(f"\nTraining Model 3 (LSTM) for {epochs} epochs...")
# YOUR CODE HERE: Train 'model_lstm' similarly to model_rnn.
# Store the output history in 'history_lstm'.
history_lstm = # ... YOUR CODE ...

print("\nAll models trained successfully! 🎉")

### 1d. Evaluate Models on Test Data


In [ ]:
# === Evaluate Models on Test Set ===
print("\nEvaluating models on the test set...")

# YOUR CODE HERE: Evaluate 'model_rnn' on X_test and y_test. Store the result in 'mse_rnn'.
# Set verbose=0 to avoid printing evaluation progress.
mse_rnn = # ... YOUR CODE ...
print(f"Model 1 (SimpleRNN) - Test MSE: {mse_rnn:.6f}")

# YOUR CODE HERE: Evaluate 'model_gru' on X_test and y_test. Store the result in 'mse_gru'.
mse_gru = # ... YOUR CODE ...
print(f"Model 2 (GRU) - Test MSE: {mse_gru:.6f}")

# YOUR CODE HERE: Evaluate 'model_lstm' on X_test and y_test. Store the result in 'mse_lstm'.
mse_lstm = # ... YOUR CODE ...
print(f"Model 3 (LSTM) - Test MSE: {mse_lstm:.6f}")

# Calculate Root Mean Squared Error (RMSE) for better interpretability (optional)
rmse_rnn = np.sqrt(mse_rnn)
rmse_gru = np.sqrt(mse_gru)
rmse_lstm = np.sqrt(mse_lstm)

print(f"\nModel 1 (SimpleRNN) - Test RMSE: {rmse_rnn:.6f} (scaled units)")
print(f"Model 2 (GRU) - Test RMSE: {rmse_gru:.6f} (scaled units)")
print(f"Model 3 (LSTM) - Test RMSE: {rmse_lstm:.6f} (scaled units)")

## Exercise 2: More Evaluation & Tuning Experiments



### 2a. Plotting Learning Curves
We will visualize how training and validation loss changed during training for each model.
Define a function to plot `loss` and `val_loss` from a model's history object. Call this function for each of the three trained models.
This helps diagnose **overfitting**.

In [ ]:
def plot_loss_curves(history, model_name):
    plt.figure(figsize=(10, 5))
    # YOUR CODE HERE: Plot the training loss (history.history['loss'])
    plt.plot(# ... YOUR CODE ... , label='Train Loss')
    # YOUR CODE HERE: Plot the validation loss (history.history['val_loss'])
    plt.plot(# ... YOUR CODE ... , label='Validation Loss')
    plt.title(f'{model_name}: Training & Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error (Loss)')
    plt.legend()
    plt.grid(True)
    plt.show()

print("\nPlotting Loss Curves for Each Model...")
# YOUR CODE HERE: Call plot_loss_curves for history_rnn with model name 'SimpleRNN'.
# ... YOUR CODE ...

# YOUR CODE HERE: Call plot_loss_curves for history_gru with model name 'GRU'.
# ... YOUR CODE ...

# YOUR CODE HERE: Call plot_loss_curves for history_lstm with model name 'LSTM'.
# ... YOUR CODE ...


### 2b. Plotting Predictions vs. Actual
We will compare the models predictions on the test set against the actual stock prices visually. We can see if the models capture the overall trend, magnitude, and turning points of the actual stock price.

In [ ]:
print("\nGenerating predictions for final plot...")
# === Make Predictions ===
# YOUR CODE HERE: Predict on X_test using model_rnn, store in test_predict_rnn
test_predict_rnn = # ... YOUR CODE ...
# YOUR CODE HERE: Predict on X_test using model_gru, store in test_predict_gru
test_predict_gru = # ... YOUR CODE ...
# YOUR CODE HERE: Predict on X_test using model_lstm, store in test_predict_lstm
test_predict_lstm = # ... YOUR CODE ...

# === Inverse Transform Predictions ===
# What: Convert scaled predictions (0-1 range) back to original dollar values.
# How: Use scaler.inverse_transform().
# Why: To compare predictions directly with the actual stock prices.
# YOUR CODE HERE: Inverse transform test_predict_rnn
test_predict_rnn = # ... YOUR CODE ...
# YOUR CODE HERE: Inverse transform test_predict_gru
test_predict_gru = # ... YOUR CODE ...
# YOUR CODE HERE: Inverse transform test_predict_lstm
test_predict_lstm = # ... YOUR CODE ...

# === Inverse Transform Actual Test Data ===
# YOUR CODE HERE: Inverse transform y_test. Reshape y_test to 2D first using .reshape(-1, 1).
y_test_actual = # ... YOUR CODE ...

# === Plotting === #
plt.figure(figsize=(15, 7))

# Get the dates corresponding to the test set
test_dates = df['Date'].iloc[train_size+look_back:]

# YOUR CODE HERE: Plot the actual test data (test_dates vs y_test_actual). Label it 'Actual Netflix Close Price'.
plt.plot(# ... YOUR CODE ... )

# YOUR CODE HERE: Plot the RNN test predictions (test_dates vs test_predict_rnn). Label it 'RNN Test Prediction'. Use linestyle='--'.
plt.plot(# ... YOUR CODE ... )

# YOUR CODE HERE: Plot the GRU test predictions (test_dates vs test_predict_gru). Label it 'GRU Test Prediction'. Use linestyle='--'.
plt.plot(# ... YOUR CODE ... )

# YOUR CODE HERE: Plot the LSTM test predictions (test_dates vs test_predict_lstm). Label it 'LSTM Test Prediction'. Use linestyle='--'.
plt.plot(# ... YOUR CODE ... )

plt.title('Netflix Stock Price Prediction Comparison (Test Set)')
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

### 2c. Experiment: Stacked LSTM
We will build a deeper LSTM network with two layers. By Adding a second `LSTM(32)` layer. The *first* LSTM layer needs `return_sequences=True`.
Stacking allows the network to potentially learn more complex, hierarchical temporal patterns.

In [ ]:
print("\n--- Experiment: Stacked LSTM ---")
model_stacked_lstm = Sequential(name="Stacked_LSTM_Model")

# YOUR CODE HERE: Add the first LSTM layer with 32 units.
# Remember to set return_sequences=True and specify input_shape.
model_stacked_lstm.add(# ... YOUR CODE ...

# YOUR CODE HERE: Add the second LSTM layer with 32 units.
# return_sequences defaults to False, which is correct here.
model_stacked_lstm.add(# ... YOUR CODE ...

# YOUR CODE HERE: Add the Dense output layer with 1 unit.
model_stacked_lstm.add(# ... YOUR CODE ...

# YOUR CODE HERE: Compile the model ('adam', 'mean_squared_error').
model_stacked_lstm.compile(# ... YOUR CODE ...

model_stacked_lstm.summary()

# --- Train and Evaluate Stacked LSTM ---
print(f"\nTraining Stacked LSTM for {epochs} epochs...")
# YOUR CODE HERE: Train the 'model_stacked_lstm' using .fit().
# Use X_train, y_train, epochs, batch_size, validation_split=0.1, verbose=0.
# Store the history in 'history_stacked'.
history_stacked = # ... YOUR CODE ...

# YOUR CODE HERE: Evaluate the trained 'model_stacked_lstm' on the test set.
# Store the result in 'mse_stacked'.
mse_stacked = # ... YOUR CODE ...
print(f"Stacked LSTM - Test MSE: {mse_stacked:.6f}")

# YOUR CODE HERE: Plot the loss curves for the stacked model using plot_loss_curves().
# ... YOUR CODE ...

### 2d. Experiment: LSTM with Dropout
We are adding Dropout regularization to the simple LSTM model by Inserting a `Dropout(0.2)` layer *after* the LSTM layer and *before* the final Dense layer.
Dropout randomly deactivates a fraction of neurons during training, forcing the network to learn more robust representations and reducing the risk of overfitting.

In [ ]:
print("\n--- Experiment: LSTM with Dropout ---")
model_dropout_lstm = Sequential(name="Dropout_LSTM_Model")

# YOUR CODE HERE: Add an LSTM layer with 32 units, specifying input_shape.
model_dropout_lstm.add(# ... YOUR CODE ...

# YOUR CODE HERE: Add a Dropout layer with a rate of 0.2 (20% dropout).
model_dropout_lstm.add(# ... YOUR CODE ...

# YOUR CODE HERE: Add the Dense output layer with 1 unit.
model_dropout_lstm.add(# ... YOUR CODE ...

# YOUR CODE HERE: Compile the model ('adam', 'mean_squared_error').
model_dropout_lstm.compile(# ... YOUR CODE ...

model_dropout_lstm.summary()

# --- Train and Evaluate Dropout LSTM ---
print(f"\nTraining LSTM with Dropout for {epochs} epochs...")
# YOUR CODE HERE: Train 'model_dropout_lstm' using .fit().
# Store the history in 'history_dropout'.
history_dropout = # ... YOUR CODE ...

# YOUR CODE HERE: Evaluate the trained 'model_dropout_lstm' on the test set.
# Store the result in 'mse_dropout'.
mse_dropout = # ... YOUR CODE ...
print(f"LSTM with Dropout - Test MSE: {mse_dropout:.6f}")

# YOUR CODE HERE: Plot the loss curves for the dropout model.
# ... YOUR CODE ...

### 2e. Experiment: Different Lookback Period
Re-train the best performing model (likely LSTM or GRU) using a different `look_back` period.
Repeat the sequence creation, reshaping, and train/test split steps with `look_back = 30`. Then define, compile, train, and evaluate the chosen model architecture again.
The choice of lookback period is a hyperparameter. A shorter period (30 days) might focus more on recent trends, while a longer one (60 days) captures more historical context. This experiment helps understand how sensitive the model is to this choice.

In [ ]:
print("\n--- Experiment: Shorter Lookback (30 days) with LSTM ---")
look_back_short = 30

# YOUR CODE HERE: Re-create sequences using 'dataset_scaled' and 'look_back_short'. Store in X_short, y_short.
X_short, y_short = # ... YOUR CODE ...

# YOUR CODE HERE: Reshape X_short to the 3D format [samples, timesteps, features].
X_short = # ... YOUR CODE ...

# YOUR CODE HERE: Perform train/test split on X_short, y_short (80/20 split).
# Use the same train_size_percentage. Store results in X_train_short, X_test_short, y_train_short, y_test_short.
train_size_short = # ... YOUR CODE ...
X_train_short, X_test_short = # ... YOUR CODE ...
y_train_short, y_test_short = # ... YOUR CODE ...

print("Short Lookback Shapes:")
print("X_train_short:", X_train_short.shape, "y_train_short:", y_train_short.shape)
print("X_test_short:", X_test_short.shape, "y_test_short:", y_test_short.shape)

# --- Build, Compile, Train, Evaluate LSTM with Short Lookback ---
model_lstm_short = Sequential(name="LSTM_ShortLookback_Model")
# YOUR CODE HERE: Add an LSTM layer with 32 units. The input_shape will use look_back_short.
model_lstm_short.add(# ... YOUR CODE ...
# YOUR CODE HERE: Add the Dense output layer.
model_lstm_short.add(# ... YOUR CODE ...

# YOUR CODE HERE: Compile the model.
model_lstm_short.compile(# ... YOUR CODE ...

print(f"\nTraining LSTM with {look_back_short}-day Lookback...")
# YOUR CODE HERE: Train 'model_lstm_short' using the _short datasets.
# Store history in 'history_lstm_short'. Use verbose=0.
history_lstm_short = # ... YOUR CODE ...

# YOUR CODE HERE: Evaluate 'model_lstm_short' on the _short test set.
# Store mse in 'mse_lstm_short'.
mse_lstm_short = # ... YOUR CODE ...
print(f"LSTM with {look_back_short}-day Lookback - Test MSE: {mse_lstm_short:.6f}")

# YOUR CODE HERE: Plot the loss curves for this short lookback model.
# ... YOUR CODE ...

## Exercise 3: Analysis and Questions

**Instructions:** Answer the following questions based on the results you obtained by running the code above. Refer back to the MSE values, loss curves, and prediction plots.

**1. Initial Model Comparison (Ex 1):** 
    * Compare the **Test MSE** results from the SimpleRNN, GRU, and LSTM models (from section 1d).
    * Which model performed best (lowest Test MSE)? Which performed worst?
    * Does this outcome align with the theoretical advantages of LSTMs/GRUs (handling long-term dependencies better than SimpleRNNs)? Explain briefly.

*(Your Answer Here)*

**2. Prediction Plot Analysis (Ex 2b):** 
    * Looking at the plot comparing actual vs. predicted prices on the test set, which model's predictions (dashed lines) visually seemed to follow the actual price trend most closely?
    * Does the model that *looks* best on the plot necessarily have the lowest Test MSE you recorded in Ex 1d? Discuss any similarities or differences between your visual assessment and the MSE scores.

*(Your Answer Here)*

**3. Task Type & Configuration:**
    * Why did we use `mean_squared_error` as the loss function for predicting stock prices, instead of a classification loss like `categorical_crossentropy`?
    * Why does the final `Dense` layer in all models have only 1 unit and use a `linear` (default) activation function?

*(Your Answer Here)*

**4. Input Shape Explanation:** For the input shape `[samples, timesteps, features]` used in this assignment:
    * What does the `samples` dimension represent (how many sequences did we create)?
    * What does the `timesteps` dimension represent (what value did we set for `look_back` initially)?
    * What does the `features` dimension represent (how many features did we use, and what was it)?

*(Your Answer Here)*

**5. Stacked LSTM (`return_sequences=True`) (Ex 2c):**
    * Explain *why* setting `return_sequences=True` on the *first* LSTM layer was necessary for the stacked model.
    * What is the shape of the output from an LSTM layer when `return_sequences=True` vs. `return_sequences=False` (the default)?

*(Your Answer Here)*

**6. Tuning Results Analysis (Ex 2c, 2d):** 
    * Compare the **Test MSE** of the simple LSTM (Ex 1d), the Stacked LSTM (Ex 2c), and the LSTM with Dropout (Ex 2d).
    * Did stacking LSTM layers improve performance over the single LSTM layer in this case? Suggest a reason why it might have helped or hurt.
    * Did adding Dropout improve performance over the single LSTM layer? Look at the loss curves for the Dropout model and the simple LSTM - does Dropout appear to reduce overfitting (i.e., is the gap between train and validation loss smaller)? Explain.

*(Your Answer Here)*

**7. Lookback Period Analysis (Ex 2e):**
    * Compare the **Test MSE** of the LSTM model with the original lookback (60 days, from Ex 1d) and the LSTM model with the shorter lookback (30 days, from Ex 2e).
    * Which lookback period resulted in better performance on the test set?
    * Why might a shorter or longer lookback period be better for predicting stock prices?

*(Your Answer Here)*

## 4. Deliverables

Please submit:
1.  This completed Jupyter Notebook file (`.ipynb`) with all code cells executed and outputs (including plots and MSE values) clearly visible. **(25% penalty if submitted notebok is not executed)** 
2.  An exported HTML version or PDF version of this notebook (`File -> Download as -> HTML`). **(25% penalty if HTML or PDF is not submitted)**